Here i'm gonna compare the difference between this each image and the last one to detect if there is a pause in the video.

First i'm gonna select a test video that has an easy to spot pause

In [ ]:
import matplotlib.pyplot as plt
import cv2


import scipy

from pathlib import Path

import numpy as np

import h5py
import math


import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

#import datashader as ds


import holoviews as hv
from holoviews import opts
from holoviews.plotting.util import process_cmap
from bokeh.palettes import Viridis256
from holoviews.operation.datashader import datashade, shade, dynspread
hv.extension('bokeh')


import platform

import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../../..")


from pathlib import Path

import cv2

import json

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral11
from bokeh.io import output_notebook
import iqplot
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import LinearColorMapper, ColorBar

from Utilities.Utils import *
from Utilities.Processing import *
from Utilities.Ballpushing_utils import *


In [ ]:

# Data loading
# Get the DataFolder

if platform.system() == "Darwin":
    DataPath = Path("/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos")
# Linux Datapath
if platform.system() == "Linux":
    DataPath = Path("/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos")

print(DataPath)
Folders = []
for folder in DataPath.iterdir():
    minfolder = str(folder).lower()
    if "tnt" in minfolder and "tracked" in minfolder and "pm" in minfolder:
        Folders.append(folder)

Folders
SavePath = Path("/mnt/labserver/DURRIEU_Matthias/Pictures/RasterPlots/")



In [ ]:
Dataset = generate_dataset(Folders, xvals=True)
Dataset.head()

In [ ]:
# Get a particular fly video from the dataset

TestFly = DataPath/'230803_TNTscreen_PM_1_Videos_Tracked/arena2/corridor3/corridor3.mp4'


In [ ]:

DataFly = Dataset[(Dataset['Date'] == '230803') & (Dataset['arena'] == 'arena2') & (Dataset['corridor'] == 'corridor3')]

In [ ]:
import cv2
import numpy as np

# Open the video file
cap = cv2.VideoCapture(TestFly.as_posix())

# Initialize a variable to store the previous frame
prev_frame = None

# Initialize a list to store the sum of pixel differences for each frame
pixel_diffs = []

while True:
    # Read the next frame from the video
    ret, frame = cap.read()

    # If the frame was read successfully
    if ret == True:
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # If this is not the first frame
        if prev_frame is not None:
            # Compute the absolute difference between the current and previous frame
            diff = cv2.absdiff(gray, prev_frame)

            # Sum up all pixel differences and append to the list
            pixel_diffs.append(np.sum(diff))

        # Save the current frame for use in the next iteration
        prev_frame = gray

    else:
        # If there was an error reading the frame, we break the loop
        break

# Release the video file
cap.release()



In [ ]:
# plot the pixel differences

curve_diff = hv.Curve(pixel_diffs, 'Frame', 'Sum of pixel differences')

curve_fly = hv.Curve(DataFly, 'Frame', 'yfly_smooth')

curve_diff.opts(width=600, height=400, tools=['hover'], show_grid=True)
curve_fly.opts(width=600, height=400, tools=['hover'], show_grid=True)

curve_diff + curve_fly

In [ ]:
147*30

In [ ]:
# Add a column to the dataframe to store where the fly is pausing

# Calculate the difference between consecutive frames
Dataset['yfly_diff'] = Dataset['yfly_smooth'].diff().abs()

# Identify periods where the difference is less than 5 for at least 200 frames
Dataset['Pausing'] = (Dataset['yfly_diff'] < 0.05).rolling(200).sum() == 200

# Replace NaN values with False
Dataset['Pausing'].fillna(False, inplace=True)

In [ ]:
Dataset.head()

In [ ]:
DataFly = Dataset[(Dataset['Date'] == '230803') & (Dataset['arena'] == 'arena2') & (Dataset['corridor'] == 'corridor3')]

In [ ]:
# Create a new column 'PauseGroup' where change in 'Pausing' is detected
DataFly['PauseGroup'] = (DataFly['Pausing'] != DataFly['Pausing'].shift()).cumsum()

# Filter rows where 'Pausing' is True
pauses = DataFly[DataFly['Pausing'] == True]

# Group by 'PauseGroup' and get the first and last frame of each pause event
pause_events = pauses.groupby('PauseGroup')['time'].agg(['first', 'last'])

# Reset the index of the DataFrame
pause_events.reset_index(drop=True, inplace=True)

# Print the pause events
print(pause_events)

# Convert them to 


In [ ]:
# Convert 'first' and 'last' columns to datetime format
pause_events['first'] = pd.to_datetime(pause_events['first'], unit='s').dt.time
pause_events['last'] = pd.to_datetime(pause_events['last'], unit='s').dt.time

# Print the pause events
print(pause_events)

In [ ]:
pauses

In [ ]:
pause_events

In [ ]:
yflydiff = hv.Curve(DataFly, 'Frame', 'yfly_diff')

yflydiff.opts(width=600, height=400, tools=['hover'], show_grid=True)

In [ ]:
8700/30

In [ ]:
10800/30

In [ ]:
 200/30

In [ ]:
def process_videos(Dataset, vidpath, OutFolder, vidname):
    
    # Add a column to the dataframe to store where the fly is pausing

    # Calculate the difference between consecutive frames
    Dataset['yfly_diff'] = Dataset['yfly_smooth'].diff().abs()

    # Identify periods where the difference is less than 5 for at least 200 frames
    Dataset['Pausing'] = (Dataset['yfly_diff'] < 0.05).rolling(200).sum() == 200

    # Replace NaN values with False
    Dataset['Pausing'].fillna(False, inplace=True)
    # Create a new column 'PauseGroup' where change in 'Pausing' is detected
    Dataset['PauseGroup'] = (Dataset['Pausing'] != Dataset['Pausing'].shift()).cumsum()

    # Filter rows where 'Pausing' is True
    pauses = Dataset[Dataset['Pausing'] == True]

    # Group by 'PauseGroup' and get the first and last frame of each pause event
    pause_events = pauses.groupby('PauseGroup')['Frame'].agg(['first', 'last'])

    # Reset the index of the DataFrame
    pause_events.reset_index(drop=True, inplace=True)

    # Print the pause events
    print(pause_events)

    clips = []
    for i, (start_frame, end_frame) in enumerate(pause_events.itertuples(index=False)):
        # Load the video
        cap = cv2.VideoCapture(str(vidpath))

        # Get the video's width, height, and frames per second (fps)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        
        # Rotate frame 90 degrees clockwise
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

        # Write some Text
        font = cv2.FONT_HERSHEY_SIMPLEX
        text = f"pause:{i+1} - start:{start_time_str}"
        font_scale = width / 150
        thickness = int(4 * font_scale)
        text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)

        # Position the text at the top center of the frame
        text_x = (frame.shape[1] - text_size[0]) // 2
        text_y = 25
        cv2.putText(
            frame,
            text,
            (text_x, text_y),
            font,
            font_scale,
            (255, 255, 255),
            thickness,
            cv2.LINE_AA,
        )

        # Define the codec and create a VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Be sure to use lower case

        clip_path = OutFolder.joinpath(f"output_{i}.mp4").as_posix()

        out = cv2.VideoWriter(
            clip_path, fourcc, fps, (height, width)
        )  # Note that width and height are swapped

        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        for _ in range(start_frame, end_frame):
            ret, frame = cap.read()
            if not ret:
                break

            # Write the frame into the output file
            out.write(frame)

        # Release everything when done
        cap.release()
        out.release()

        clips.append(clip_path)

    cv2.destroyAllWindows()

    # Define the codec and create a VideoWriter object for the final video
    out = cv2.VideoWriter(
        OutFolder.joinpath(f"{vidname}.mp4").as_posix(), fourcc, fps, (height, width)
    )  # Note that width and height are swapped because the clips were rotated

    # Assuming 'clips' is a list of paths to the clip files
    for clip_path in clips:
        cap = cv2.VideoCapture(clip_path)

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Write the frame into the final output file
            out.write(frame)

        cap.release()

    # Release the final output file when done
    out.release()

    # Delete the clips
    for clip_path in clips:
        os.remove(clip_path)

    print(f"Finished processing {vidname}!")


In [ ]:
SaveVideo = Path("/mnt/labserver/DURRIEU_Matthias/Videos/TestPauses/")

process_videos(DataFly, TestFly, SaveVideo, 'TestFly')

# Using my function

In [ ]:
TestFly

In [ ]:
flypath = next(TestFly.parent.glob("*flytrack*.analysis.h5"))

In [ ]:
flypath

In [ ]:
pause_events = extract_pauses(flypath)

In [ ]:
pause_events

# Adding x fly position

In [ ]:
DataFly['yfly_diff'] = Dataset['yfly_smooth'].diff().abs()
DataFly['xfly_diff'] = Dataset['xfly_smooth'].diff().abs()

In [ ]:
# plot the test fly x diff position to check pauses

xflydiff = hv.Curve(DataFly, 'Frame', 'xfly_diff')

xflydiff.opts(width=600, height=400, tools=['hover'], show_grid=True)

In [ ]:
flypath

In [ ]:
flypath = next(TestFly.parent.glob("*flytrack*.analysis.h5"))

#flypath = str(flypath)

pause_events = extract_pauses(flypath, min_time = 200, threshold_x=1, threshold_y=0.05)

pause_events


In [ ]:
pause_events[0]['time']

In [ ]:
import datetime

# Initialize an empty list to store the first and last times
first_last_times = []

# Iterate over the list of DataFrames
for df in pause_events:
    # Check if the DataFrame is not empty
    if not df.empty:
        # Get the first and last 'time' values
        first_time = df['time'].iloc[0]
        last_time = df['time'].iloc[-1]
        
        # Convert the times from seconds to 'h:m:s' format
        first_time_str = str(datetime.timedelta(seconds=int(first_time)))
        last_time_str = str(datetime.timedelta(seconds=int(last_time)))
        
        # Append the times to the list
        first_last_times.append((first_time_str, last_time_str))

# Print the first and last times
for i, (first, last) in enumerate(first_last_times):
    print(f"Pause event {i+1}: First time = {first}, Last time = {last}")


In [ ]:
pause_events_bounds = []
for df in pause_events:
    first_time = df['time'].iloc[0]
    last_time = df['time'].iloc[-1]
    pause_events_bounds.append((first_time, last_time))

from datetime import timedelta

pause_events_hms = []
for first_time, last_time in pause_events_bounds:
    first_time_hms = str(timedelta(seconds=int(first_time)))
    last_time_hms = str(timedelta(seconds=int(last_time)))
    pause_events_hms.append((first_time_hms, last_time_hms))



In [ ]:
pause_events_hms